In [22]:
version = "v1"

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elsanns/xai-nlp-notebooks/blob/master/fine_tune_bart_summarization_two_langs.ipynb)
[GitHub](https://github.com/elsanns/xai-nlp-notebooks/blob/master/fine_tune_bart_summarization_two_langs.ipynb)
# Fine-tuning BART for summarization in two languages

---

In a world of ever-growing amount of data, the task of automatically creating coherent and fluent summaries is gaining importance. Coming up with a shorter, concise version of a document, can help to derive value from large volumes of text.

This notebook contains an example of fine-tuning [Bart](https://huggingface.co/transformers/model_doc/bart.html) for generating summaries of article sections from the [WikiLingua](https://huggingface.co/datasets/wiki_lingua) dataset. WikiLingua is a multilingual set of articles. We will run the same code for two Bart checkpoints, including a non-English model from the [Hugging Face Model Hub](https://huggingface.co/models). We will be using:
- the **English** portion of WikiLingua with [sshleifer/distilbart-xsum-12-3](https://huggingface.co/sshleifer/distilbart-xsum-12-3) Bart checkpoint, and
- **French** articles from the WikiLingua with the [moussaKam/barthez-orangesum-abstract](https://huggingface.co/moussaKam/barthez-orangesum-abstract) model.

## Setup

---

In [2]:
# ! pip install transformers
# ! pip install datasets
# ! pip install sentencepiece
# ! pip install rouge_score
# ! pip install wandb

In [1]:
import torch
import numpy as np
import datasets

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)

from tabulate import tabulate
import nltk
from datetime import datetime

In [2]:
datasets.disable_progress_bar()

In [3]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    import wandb

    wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vimig-socrates. Use `wandb login --relogin` to force relogin


In [4]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.", flush=True)


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}", flush=True)
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}", flush=True)
    print_gpu_utilization()

## Model and tokenizer

---

Download model and tokenizer. Use default parameters or try custom values (see [HF Bart configuration](https://huggingface.co/transformers/_modules/transformers/configuration_bart.html) and [Fairseq Bart](https://github.com/pytorch/fairseq/tree/master/examples/bart)).

In [5]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("GanjinZero/biobart-large")
# model = AutoModelForSeq2SeqLM.from_pretrained("GanjinZero/biobart-large")

model_name = "GanjinZero/biobart-large"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set model parameters or use the default
# print(model.config)

# tokenization
encoder_max_length = 1024  # demo
decoder_max_length = 1024

## Data

---

### Download

For demonstration, we are only using a small portion of the data.

In [7]:
import pandas as pd
from datasets import Dataset


In [8]:
print_gpu_utilization()

GPU memory occupied: 93 MB.


In [9]:
train_data = pd.read_pickle("/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/train/discharge_target_with_preceding_text+structured_data.pickle")
valid_data = pd.read_pickle("/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/valid/discharge_target_with_preceding_text+structured_data.pickle")

In [10]:
train_ds = Dataset.from_pandas(train_data[['hadm_id', "bhc_preceding_text", "brief_hospital_course"]], split="train")
valid_ds = Dataset.from_pandas(valid_data[['hadm_id', "bhc_preceding_text", "brief_hospital_course"]], split="valid")

In [11]:
# data = datasets.load_dataset("wiki_lingua", name=language, split="train[:200]")

# Take a look at the data
# print(train_ds[0])

# data['article'][0]

### Prepare

**Format and split into train and validation sets**

In [12]:
def flatten(example):
    return {
        "document": example["bhc_preceding_text"],
        "summary": example["brief_hospital_course"],
    }


def list2samples(example):
    documents = []
    summaries = []
    for sample in zip(example["document"], example["summary"]):
        if len(sample[0]) > 0:
            documents += sample[0]
            summaries += sample[1]
    return {"document": documents, "summary": summaries}


train_dataset_txt = train_ds.map(flatten, remove_columns=['hadm_id', "bhc_preceding_text", "brief_hospital_course"])
# we don't need to do this step because we don't have multiple doc/summary pairs within each example
# train_dataset = train_dataset.map(list2samples, batched=True)

valid_dataset_txt = valid_ds.map(flatten, remove_columns=['hadm_id', "bhc_preceding_text", "brief_hospital_course"])
# valid_dataset = valid_dataset.map(list2samples, batched=True)

# train_data_txt, validation_data_txt = dataset.train_test_split(test_size=0.1).values()

**Preprocess and tokenize**

In [13]:
# We already did this, so we can just load this dataset

# def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
#     source, target = batch["document"], batch["summary"]
#     source_tokenized = tokenizer(
#         source, padding="max_length", truncation=True, max_length=max_source_length
#     )
#     target_tokenized = tokenizer(
#         target, padding="max_length", truncation=True, max_length=max_target_length
#     )

#     batch = {k: v for k, v in source_tokenized.items()}
#     # Ignore padding in the loss
#     batch["labels"] = [
#         [-100 if token == tokenizer.pad_token_id else token for token in l]
#         for l in target_tokenized["input_ids"]
#     ]
#     return batch


# train_data = train_dataset_txt.map(
#     lambda batch: batch_tokenize_preprocess(
#         batch, tokenizer, encoder_max_length, decoder_max_length
#     ),
#     batched=True,
#     remove_columns=train_dataset_txt.column_names,
# )

# validation_data = valid_dataset_txt.map(
#     lambda batch: batch_tokenize_preprocess(
#         batch, tokenizer, encoder_max_length, decoder_max_length
#     ),
#     batched=True,
#     remove_columns=valid_dataset_txt.column_names,
# )

In [14]:
# train_data.save_to_disk("/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/train/discharge_target_with_preceding_text_BioBART_data.hf")
# validation_data.save_to_disk("/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/valid/discharge_target_with_preceding_text_BioBART_data.hf")

In [15]:
from datasets import load_from_disk
train_data = load_from_disk("/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/train/discharge_target_with_preceding_text_BioBART_data.hf")
validation_data = load_from_disk("/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/valid/discharge_target_with_preceding_text_BioBART_data.hf")

In [16]:
print("After processing datasets", flush=True)
print_gpu_utilization()

After processing datasets
GPU memory occupied: 93 MB.


## Training

---

### Metrics

In [17]:
# Borrowed from https://github.com/huggingface/transformers/blob/master/examples/seq2seq/run_summarization.py

nltk.download("punkt", quiet=True)

metric = datasets.load_metric("rouge")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

/tmp/ipykernel_3669193/4242988070.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("rouge")
/home/vs428/.conda/envs/peft_finetune_env/lib/python3.12/site-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


### Training arguments

In [18]:
# epochs = 2
# train_size = train_data.num_rows
# train_batch_size = 8
# ga_steps = 1
# virtual_batch_size = train_batch_size * ga_steps   # "invented name" => 256
# per_epoch_steps = int(train_size / virtual_batch_size + 0.5) # round => 121
# total_steps = epochs * per_epoch_steps # => 605

In [23]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"bart-dischargeme-results_{version}",
    num_train_epochs=2,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=8,  # demo
    per_device_eval_batch_size=2,
    # learning_rate=3e-05,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir=f"bart-dischargeme-logs_{version}",
    logging_steps=5,
    save_total_limit=2,

    save_strategy="steps",
    evaluation_strategy="steps",
    eval_steps=250,
    save_steps=500,
    fp16=True,
    push_to_hub=False,
    metric_for_best_model='eval_loss',
    load_best_model_at_end=True,
    greater_is_better=False,
    disable_tqdm=True,
    log_level="info",
    logging_first_step=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data.shuffle(seed=42).select(range(2000)),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 5)]
)

/home/vs428/.conda/envs/peft_finetune_env/lib/python3.12/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Using auto half precision backend


In [24]:
print("============== After setting up trainer and args ==============", flush=True)
print_gpu_utilization()


============== After setting up trainer and args ==============
GPU memory occupied: 1791 MB.


### Train

Wandb integration

In [25]:
if WANDB_INTEGRATION:
    wandb_run = wandb.init(
        project="bart-dischargeme",
        config={
            "per_device_train_batch_size": training_args.per_device_train_batch_size,
            "learning_rate": training_args.learning_rate,
            "dataset": "dischargeme preceding_text_only",
        },
    )

    now = datetime.now()
    current_time = now.strftime("%H%M%S")
    wandb_run.name = "run_" + current_time

Evaluate before fine-tuning

In [26]:
print("============== Before trainer.evaluate() pretrain ==============", flush=True)
print_gpu_utilization()


============== Before trainer.evaluate() pretrain ==============
GPU memory occupied: 2086 MB.


In [27]:
# trainer.evaluate()

In [28]:
print("============== After trainer.evaluate() pretrain ==============", flush=True)
print_gpu_utilization()


============== After trainer.evaluate() pretrain ==============
GPU memory occupied: 2086 MB.


Train the model

In [29]:
#%%wandb
# uncomment to display Wandb charts

trainer.train()

Step,Training Loss,Validation Loss


/home/vs428/.conda/envs/peft_finetune_env/lib/python3.12/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
print("============== After trainer.train() ==============", flush=True)
print_gpu_utilization()


Evaluate after fine-tuning

In [ ]:
trainer.evaluate()

In [ ]:
if WANDB_INTEGRATION:
    wandb_run.finish()

## Evaluation

---

**Generate summaries from the fine-tuned model and compare them with those generated from the original, pre-trained one.**

In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained("/home/vs428/Documents/DischargeMe/hail-dischargeme/notebooks/brief_hospital_course/template_code/bart-dischargeme-results_v2/checkpoint-16500/")

In [ ]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["document"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)

test_samples = valid_dataset_txt.select(range(16))

x, summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
y, summaries_after_tuning = generate_summary(test_samples, model)[1]

loading configuration file config.json from cache at /home/vs428/.cache/huggingface/hub/models--GanjinZero--biobart-large/snapshots/47c968d3aba260d5737fda6f5b2ad3aa35ce75e8/config.json
Model config BartConfig {
  "_name_or_path": "GanjinZero/biobart-large",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_

In [25]:
summaries_after_tuning

['Ms. ___ is a ___ year old lady with lupus \nanticoag',
 '___ year old man with a history of complete heart \nblock s/p pacemaker',
 '___ is a pleasant ___ yo woman with \nmedical history of HTN who was transferred',
 'The patient was admitted to the General Surgical \nService on ___ for evaluation and treatment',
 'Mr. ___ is a ___ w/ hx of lumbar \nstenosis',
 'Mr. ___ is a ___ year old man with HTN who \npresented with',
 'Mr. ___ is an ___ with multiple medical \ncomorbidities including ische',
 'Ms. ___ is a ___ female with past \nmedical history of uveitis who',
 'Ms. ___ is a ___ year old female with a past \nmedical history of depression',
 'Ms. ___ is a ___ ___ speaking female with a \nhistory of benign parox',
 '___ with Parkinsons disease (nonverbal at \nbaseline), HTN,',
 '___ F with hx depression, HTN, alpha thalassemia trait \n',
 'Mr. ___ is a ___ year old man with a history \nof atrial f',
 '___ yo female with history of CAD s/p stent placement x 7, \n',
 '___ yo F wit

In [21]:
print(
    tabulate(
        zip(
            range(len(summaries_after_tuning)),
            summaries_after_tuning,
            summaries_before_tuning,
        ),
        headers=["Id", "Summary after", "Summary before"],
    )
)
print("\nTarget summaries:\n")
print(
    tabulate(list(enumerate(test_samples["summary"])), headers=["Id", "Target summary"])
)
print("\nSource documents:\n")
print(tabulate(list(enumerate(test_samples["document"])), headers=["Id", "Document"]))

  Id  Summary after                                               Summary before
----  ----------------------------------------------------------  ------------------------------------------------------------------------
   0  Ms. ___ is a ___ year old lady with lupus                   .Father still alive.in 1999.Family still alive
      anticoag
   1  ___ year old man with a history of complete heart           Name:  ___   ,  .   -
      block s/p pacemaker
   2  ___ is a pleasant ___ yo woman with                         D.   5 - 5   3.2.2
      medical history of HTN who was transferred
   3  The patient was admitted to the General Surgical            Name:  ___   .   (   )
      Service on ___ for evaluation and treatment
   4  Mr. ___ is a ___ w/ hx of lumbar                            Social-icteric History:  ___   .   I   ___
      stenosis
   5  Mr. ___ is a ___ year old man with HTN who                  ف.Name:  ___
      presented with
   6  Mr. ___ is an ___ with multiple med